# Funciones

Vamos a crear un dataframe para cada endpoint de la API que estamos construyendo. Estos dataframes se almacenarán en un archivo **.parquet** que será utilizado en las llamadas a la API, con el objetivo de mejorar el rendimiento.

Importamos librerias a utilizar

In [18]:
import pandas as pd
from datetime import datetime
import scipy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


Cargamos los dataframe

In [21]:
df_steam_games = pd.read_csv('../Data/Data-Limpia/steam_games.csv')
df_user_reviews = pd.read_csv('../Data/Data-Limpia/user_reviews.csv')
df_users_items = pd.read_csv('../Data/Data-Limpia/users_items.csv')

### 1. Funcion PlayTimeGenre

##### Dataframe para Endpoint

In [22]:
df_users_items.head(3)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0


Vamos a agrupar los datos por nombre del juego y sumar las horas totales jugadas para cada juego.

In [23]:
horas_jugadas = df_users_items.groupby('item_name')['playtime_forever'].sum().reset_index()
horas_jugadas.head(3)

,item_name,playtime_forever
0,神明的一天世界(God's One Day World),6969.0
1,! That Bastard Is Trying To Steal Our Gold !,742.0
2,"""Glow Ball"" - The billiard puzzle game",21.0


Eliminemos las filas en el dataframe de steam_games que tengan valores nulos en 'title' debido a que sin ese valor no podemos saber a que juego hace referencia la fila.

In [24]:
df_steam_games.dropna(subset=['title'], inplace=True)
df_steam_games.head(3)

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id,developer
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,Strategy,Single-player,4.99,False,761140,Kotoshiro
1,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,Action,Single-player,4.99,False,761140,Kotoshiro
2,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,Indie,Single-player,4.99,False,761140,Kotoshiro


Vamos a unir los dataframe de horas jugadas con el dataframe de juegos de steam a traves de las columnas **'title** y **'item_name'** para obtener un dataframe de horas jugadas por genero.

In [25]:
df_generos = pd.merge(df_steam_games, horas_jugadas, left_on='title',right_on='item_name', how='inner')
df_generos.head(3)

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id,developer,item_name,playtime_forever
0,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,1997-06-30,Racing,Single-player,9.99,False,282010,Stainless Games Ltd,Carmageddon Max Pack,9319.0
1,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,1997-06-30,Racing,Multi-player,9.99,False,282010,Stainless Games Ltd,Carmageddon Max Pack,9319.0
2,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,1997-06-30,Racing,Steam Trading Cards,9.99,False,282010,Stainless Games Ltd,Carmageddon Max Pack,9319.0


Eliminamos duplicados y columnas que no necesitaremos para el endpoint

In [26]:
df_generos.drop(['publisher','app_name','tags','specs','price','early_access','developer','item_name'],axis=1, inplace=True)
df_generos.drop_duplicates(inplace=True)
df_generos.head(3)

,genres,title,release_date,id,playtime_forever
0,Action,Carmageddon Max Pack,1997-06-30,282010,9319.0
24,Indie,Carmageddon Max Pack,1997-06-30,282010,9319.0
48,Racing,Carmageddon Max Pack,1997-06-30,282010,9319.0


In [27]:
df_generos.reset_index(drop=True, inplace=True)

Vamos a covertir a formato de fecha toda la columna **'release_date** que cumplan el formato "YYYY-MM-DD"

In [29]:
df_generos['release_date'] = pd.to_datetime(df_generos['release_date'], format='mixed', errors='coerce')

Agregamos una columna con el año de lanzamiento

In [30]:
df_generos['release_year'] = df_generos['release_date'].dt.year
df_generos.drop(['title', 'release_date', 'id'],axis=1, inplace=True)
df_generos.head(3)

,genres,playtime_forever,release_year
0,Action,9319.0,1997.0
1,Indie,9319.0,1997.0
2,Racing,9319.0,1997.0


Alamacenamos el nuevo dataframe en un archivo parquet

In [14]:
df_generos.to_parquet('../generos.parquet')

##### Funcion

In [32]:
def PlayTimeGenre(genero: str):
    try:
        # Leer el dataframe desde el archivo parquet
        df = pd.read_parquet('../generos.parquet')

        # Obtener la lista de géneros válidos
        generos_validos = list(df['genres'].drop_duplicates())

        # Validar si el género proporcionado está en la lista de géneros válidos
        if genero.capitalize() not in generos_validos:
            return f"El género '{genero}' no es válido."
        
        # Filtrar el dataframe por el género especificado
        df = df[df['genres'] == genero.capitalize()]

        # Calcular la suma de horas jugadas por año para el género especificado
        df_agrupado = df.groupby('release_year')['playtime_forever'].sum().reset_index()

        # Encontrar el año con más horas jugadas para el género especificado
        año_mas_horas = df_agrupado.loc[df_agrupado['playtime_forever'].idxmax()]['release_year']

        # Construir la respuesta JSON con el año de lanzamiento con más horas jugadas
        content = { f"Año de lanzamiento con más horas jugadas para Género {genero.capitalize()}" : f"{año_mas_horas}" }

        return content
    
    except Exception as e:
        # Capturar y retornar un mensaje genérico en caso de otros errores
        content={"Error": f"Ocurrió un error al procesar la solicitud"}
        return content

In [36]:
PlayTimeGenre('Racing')

{'Año de lanzamiento con más horas jugadas para Género Racing': '2016.0'}

### 2. Funcion UserForGenre

##### Dataframe para Endpoint

In [31]:
df_aux = df_steam_games[['genres','id','release_date']].copy()
df_aux.drop_duplicates(inplace=True)

In [32]:
df_users_generos = pd.merge(df_aux,df_users_items, left_on='id', right_on='item_id', how='inner')
df_users_generos.head(3)

,genres,id,release_date,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,Action,282010,1997-06-30,UTNerd24,188,76561198053985682,http://steamcommunity.com/id/UTNerd24,282010,Carmageddon Max Pack,5.0,0.0
1,Action,282010,1997-06-30,I_DID_911_JUST_SAYING,154,76561198067520555,http://steamcommunity.com/id/I_DID_911_JUST_SA...,282010,Carmageddon Max Pack,0.0,0.0
2,Action,282010,1997-06-30,76561197962104795,79,76561197962104795,http://steamcommunity.com/profiles/76561197962...,282010,Carmageddon Max Pack,0.0,0.0


In [35]:
df_users_generos['release_date'] = pd.to_datetime(df_users_generos['release_date'], errors='coerce')

In [36]:
df_users_generos['release_year'] = df_users_generos['release_date'].dt.year
df_users_generos.drop('release_date',axis=1,inplace=True)
df_users_generos.head(3)

,genres,id,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,release_year
0,Action,282010,UTNerd24,188,76561198053985682,http://steamcommunity.com/id/UTNerd24,282010,Carmageddon Max Pack,5.0,0.0,1997.0
1,Action,282010,I_DID_911_JUST_SAYING,154,76561198067520555,http://steamcommunity.com/id/I_DID_911_JUST_SA...,282010,Carmageddon Max Pack,0.0,0.0,1997.0
2,Action,282010,76561197962104795,79,76561197962104795,http://steamcommunity.com/profiles/76561197962...,282010,Carmageddon Max Pack,0.0,0.0,1997.0


In [39]:
df_users_generos.drop(['id','items_count','steam_id','user_url','playtime_2weeks'], axis=1, inplace=True)
df_users_generos.head(3)

,genres,user_id,item_id,item_name,playtime_forever,release_year
0,Action,UTNerd24,282010,Carmageddon Max Pack,5.0,1997.0
1,Action,I_DID_911_JUST_SAYING,282010,Carmageddon Max Pack,0.0,1997.0
2,Action,76561197962104795,282010,Carmageddon Max Pack,0.0,1997.0


In [40]:
df_users_generos.to_parquet('../users_generos.parquet', index=False)

#### Funcion

In [4]:
def UserForGenre(genero: str):
    """
    Encuentra el usuario con más horas jugadas para un género específico.

    Args:
    - genero: Género específico para el cual se desea encontrar al usuario con más horas jugadas.

    Returns:
    - Diccionario que contiene el usuario con más horas jugadas para el genero dado y las horas jugadas por año para ese género.
    """

    df = pd.read_parquet('../users_generos.parquet')
    # Convertir el género a minúsculas para realizar la búsqueda sin distinción entre mayúsculas y minúsculas
    genero_minuscula = genero.lower()

    # Filtrar el DataFrame para el género especificado (ignorando mayúsculas y minúsculas)
    df_filtered = df[df['genres'].str.lower().str.contains(genero_minuscula, na=False)]

    if df_filtered.empty:
        return f"No se encontraron datos para el género '{genero}'."

    # Agrupar por 'user_id' y 'year', sumar las horas jugadas
    df_grouped = df_filtered.groupby(['user_id', 'release_year'])['playtime_forever'].sum().reset_index()

    if df_grouped.empty:
        return f"No se encontraron datos de horas jugadas para el género '{genero}'."

    # Encontrar el usuario con la máxima suma de horas jugadas
    max_playtime_user = df_grouped.loc[df_grouped['playtime_forever'].idxmax(), 'user_id']

    # Filtrar el DataFrame para el usuario con máxima suma de horas jugadas
    df_user_max_playtime = df_grouped[df_grouped['user_id'] == max_playtime_user]

    # Crear la lista de horas jugadas por año en el formato esperado
    horas_jugadas = []
    for index, row in df_user_max_playtime.iterrows():
        horas_jugadas.append({
            'Año': int(row['release_year']),  # Convertir el año a entero
            'Horas': int(row['playtime_forever'])  # Convertir las horas a entero
        })

    # Construir el diccionario de retorno
    return {
        f"Usuario con más horas jugadas para '{genero}'": max_playtime_user,
        "Horas jugadas": horas_jugadas
    }


In [5]:
UserForGenre("Action")

{"Usuario con más horas jugadas para 'Action'": 'Evilutional',
 'Horas jugadas': [{'Año': 2003, 'Horas': 0},
  {'Año': 2006, 'Horas': 0},
  {'Año': 2009, 'Horas': 2037},
  {'Año': 2010, 'Horas': 4102},
  {'Año': 2011, 'Horas': 1968},
  {'Año': 2012, 'Horas': 680991},
  {'Año': 2013, 'Horas': 69726},
  {'Año': 2014, 'Horas': 24226},
  {'Año': 2015, 'Horas': 112},
  {'Año': 2016, 'Horas': 1291},
  {'Año': 2017, 'Horas': 10894}]}

### 6. Función recomendacion_juego

##### Dataframe para Endpoint

En nuestro modelo vamos a utilizar los conjuntos de datos **steam_games.csv** y **user_reviews.csv** y vamos a unirlo mediante el 'id_item' que corresponde al identificador único del juego. 

In [19]:
df_games = pd.read_csv('../Data/Data-Limpia/steam_games.csv')
df_reviews = pd.read_csv('../Data/Data-Limpia/user_reviews.csv')
df_modelo= pd.merge(df_games,df_reviews,left_on='id',right_on='item_id', how='inner')
df_modelo.head(3)

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id,developer,user_id,posted,item_id,helpful,recommend,sentiment_analysis
0,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,1997-06-30,Racing,Single-player,9.99,False,282010,Stainless Games Ltd,InstigatorAU,NaN,282010,No ratings yet,True,1
1,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,1997-06-30,Racing,Multi-player,9.99,False,282010,Stainless Games Ltd,InstigatorAU,NaN,282010,No ratings yet,True,1
2,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,1997-06-30,Racing,Steam Trading Cards,9.99,False,282010,Stainless Games Ltd,InstigatorAU,NaN,282010,No ratings yet,True,1


Eliminarmos las columnas que no vamos a utilizar y solo dejamos los features que será de utilidad para el modelo

In [20]:
df_modelo.drop(['publisher','app_name','release_date','early_access','id','developer','user_id','posted','helpful','price','tags','specs'], axis=1, inplace=True)
df_modelo.head(3)

,genres,title,item_id,recommend,sentiment_analysis
0,Action,Carmageddon Max Pack,282010,True,1
1,Action,Carmageddon Max Pack,282010,True,1
2,Action,Carmageddon Max Pack,282010,True,1


Contamos los sentimientos positivos, negativos y neutrales para cada juegos y la cantidad de recomendaciones y no recomendaciones para cada juego

In [21]:
# Usar pd.crosstab para contar la cantidad de cada valor de sentimiento por título
conteo_sentimientos = pd.crosstab(df_modelo['title'], df_modelo['sentiment_analysis'])
# conteo_recomendaciones = pd.crosstab(df_modelo['title'], df_modelo['recommend'])
# Renombrar las columnas para mayor claridad
conteo_sentimientos.columns = ['negative_reviews','neutral_reviews','positive_reviews']
# conteo_recomendaciones.columns = ['no_recommend_count', 'recommend_count']
# Reiniciar el índice para obtener 'title' como columna
conteo_sentimientos.reset_index(inplace=True)
# conteo_recomendaciones.reset_index(inplace=True)


Unimos los el contemos de sentimiento y recomendaciones a traves de la columna 'title' del datafram original.

In [22]:
df_modelo = pd.merge(df_modelo, conteo_sentimientos, on='title', how='inner')
# df_modelo = pd.merge(df_modelo, conteo_recomendaciones, on='title', how='inner')
df_modelo.head(3)

,genres,title,item_id,recommend,sentiment_analysis,negative_reviews,neutral_reviews,positive_reviews
0,Action,Carmageddon Max Pack,282010,True,1,0,72,0
1,Action,Carmageddon Max Pack,282010,True,1,0,72,0
2,Action,Carmageddon Max Pack,282010,True,1,0,72,0


In [23]:
# df_modelo.drop(['recommend','sentiment_analysis'], axis=1, inplace=True)
df_modelo.drop(['sentiment_analysis'], axis=1, inplace=True)
df_modelo.drop_duplicates(inplace=True)
df_modelo.reset_index(drop=True, inplace=True)
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10321 entries, 0 to 10320
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   genres            10287 non-null  object
 1   title             10321 non-null  object
 2   item_id           10321 non-null  int64 
 3   recommend         10321 non-null  bool  
 4   negative_reviews  10321 non-null  int64 
 5   neutral_reviews   10321 non-null  int64 
 6   positive_reviews  10321 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 494.0+ KB


In [26]:
text_features = ['genres','title']
# numeric_features = ['negative_reviews', 'neutral_reviews', 'positive_reviews','no_recommend_count','recommend_count']
numeric_features = ['negative_reviews', 'neutral_reviews', 'positive_reviews']

In [28]:
df_modelo['combined_text'] = df_modelo[text_features].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_modelo.drop(['genres','recommend'], axis=1, inplace=True)
df_modelo.head(3)

,title,item_id,negative_reviews,neutral_reviews,positive_reviews,combined_text
0,Carmageddon Max Pack,282010,0,72,0,Action Carmageddon Max Pack
1,Carmageddon Max Pack,282010,0,72,0,Indie Carmageddon Max Pack
2,Carmageddon Max Pack,282010,0,72,0,Racing Carmageddon Max Pack


In [29]:
df_modelo.duplicated().sum()

2611

In [31]:
df_modelo.drop_duplicates(inplace=True)
df_modelo.reset_index(drop=True, inplace=True)
df_modelo.duplicated().sum()

0

Guardamos nuestro dataset en un archivo parquet para ser utilizado en nuestro endpoint

In [33]:
df_modelo.to_parquet('../modelo.parquet', index=False)

#### Funcion

In [34]:
def recomendacion_juego(item_id: str):
    """
    Función para obtener juegos recomendados para un juego específico en una muestra aleatoria del dataframe.
    
    Args:
    - item_id: Id del juego para el cual se desean obtener recomendaciones.
    
    Returns:
    - Lista de 5 nombres de juegos recomendados únicos (sin repeticiones) que no incluye el juego dado como argumento.
    """
    df_modelo = pd.read_parquet('../modelo.parquet')

    #Inicializar CountVectorizer para convertir texto en una matriz de recuentos de términos
    count_vectorizer = CountVectorizer()
    text_matrix = count_vectorizer.fit_transform(df_modelo['combined_text'])

    #Normalizar características numéricas   
    scaler = StandardScaler()
    numeric_matrix = scaler.fit_transform(df_modelo[numeric_features])

    # Combinar matrices de características de texto y numéricas. Usamos el modulo scipy.sparse que optimiza los calculos para matrices que tienen en su mayoria ceros.
    combined_matrix = scipy.sparse.hstack((text_matrix, numeric_matrix))

    # Calcula la matriz de similitud de coseno
    cosine_sim = cosine_similarity(combined_matrix, combined_matrix)

    # Obtener el índice del juego correspondiente al nombre dado
    idx = df_modelo[df_modelo['item_id'] == item_id].index[0]
    # Calcular la similitud del juego con todos los demás juegos en la muestra
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar los juegos según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de los juegos más similares (excluyendo el juego dado)
    sim_scores = [score for score in sim_scores if df_modelo.iloc[score[0]]['item_id'] != item_id][:5]

    # Obtener los nombres de los juegos recomendados (excluyendo repeticiones)
    recommended_games = []
    seen_games = set()
    for score in sim_scores:
        game_name = df_modelo.iloc[score[0]]['title']
        if game_name not in seen_games:
            recommended_games.append(game_name)
            seen_games.add(game_name)

    return recommended_games

In [35]:
recomendacion_juego(282010)

['Kombat Pack',
 'Max Payne',
 'Max Payne 3',
 'Mad Max',
 'Max Payne 3: Deathmatch Made In Heaven Pack']